<a href="https://colab.research.google.com/github/trietp1253201581/KaggleCompetition/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dự đoán khả năng sống sót trên tàu Titanic

[Link_to_competition](https://www.kaggle.com/competitions/titanic)

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [2]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 36.7MB/s]


In [3]:
from zipfile import ZipFile
with ZipFile(file='titanic.zip') as f:
    f.extractall(path='titanic_data')

In [4]:
import pandas as pd

In [5]:
raw_df = pd.read_csv('titanic_data/train.csv')
test_df = pd.read_csv('titanic_data/test.csv')
sub_df = pd.read_csv('titanic_data/gender_submission.csv')

In [6]:
raw_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [8]:
sub_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [9]:
target_col = 'Survived'
input_cols = raw_df.columns[2:].tolist()

In [10]:
input_cols

['Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [11]:
input_cols.remove('Name')

In [12]:
input_cols.remove('Ticket')

In [13]:
raw_df[input_cols].isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [14]:
import numpy as np

In [15]:
numeric_cols = raw_df[input_cols].select_dtypes(include=np.number).columns.tolist()
categorical_cols = raw_df[input_cols].select_dtypes(include='object').columns.tolist()

In [16]:
numeric_cols.remove('Pclass')
categorical_cols.append('Pclass')

In [17]:
raw_df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [18]:
avg_age = raw_df['Age'].mean()
avg_age

29.69911764705882

In [19]:
raw_df.fillna({'Age': avg_age}, inplace=True)

In [20]:
raw_df[numeric_cols].isna().sum()

Age      0
SibSp    0
Parch    0
Fare     0
dtype: int64

In [21]:
raw_df[input_cols]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.000000,1,0,7.2500,NaN,S
1,1,female,38.000000,1,0,71.2833,C85,C
2,3,female,26.000000,0,0,7.9250,NaN,S
3,1,female,35.000000,1,0,53.1000,C123,S
4,3,male,35.000000,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
886,2,male,27.000000,0,0,13.0000,NaN,S
887,1,female,19.000000,0,0,30.0000,B42,S
888,3,female,29.699118,1,2,23.4500,NaN,S
889,1,male,26.000000,0,0,30.0000,C148,C


In [22]:
numeric_cols.remove('Age')

In [23]:
raw_df['AgeGroup'] = pd.cut(raw_df['Age'],
                            bins=[-1,5,14,18,30,60,100],
                            labels=['Baby','Child','Teenager','Adult','OldAdult','Old'])

In [24]:
raw_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,OldAdult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,OldAdult
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,OldAdult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,Adult
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,Adult
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,Adult
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,Adult


In [25]:
raw_df['Embarked'] = raw_df['Embarked'].fillna('U')

In [26]:
raw_df['Cabin'].values

array([nan, 'C85', nan, 'C123', nan, nan, 'E46', nan, nan, nan, 'G6',
       'C103', nan, nan, nan, nan, nan, nan, nan, nan, nan, 'D56', nan,
       'A6', nan, nan, nan, 'C23 C25 C27', nan, nan, nan, 'B78', nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, 'D33', nan, 'B30', 'C52', nan, nan, nan,
       nan, nan, 'B28', 'C83', nan, nan, nan, 'F33', nan, nan, nan, nan,
       nan, nan, nan, nan, 'F G73', nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, 'C23 C25 C27', nan, nan, nan, 'E31', nan,
       nan, nan, 'A5', 'D10 D12', nan, nan, nan, nan, 'D26', nan, nan,
       nan, nan, nan, nan, nan, 'C110', nan, nan, nan, nan, nan, nan, nan,
       'B58 B60', nan, nan, nan, nan, 'E101', 'D26', nan, nan, nan,
       'F E69', nan, nan, nan, nan, nan, nan, nan, 'D47', 'C123', nan,
       'B86', nan, nan, nan, nan, nan, nan, nan, nan, 'F2', nan, nan,
       'C2', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [27]:
raw_df['Cabin'] = raw_df['Cabin'].fillna('UNK').copy()

In [28]:
raw_df['Cabin'].value_counts()

Cabin
UNK            687
C23 C25 C27      4
G6               4
B96 B98          4
C22 C26          3
              ... 
E34              1
C7               1
C54              1
E36              1
C148             1
Name: count, Length: 148, dtype: int64

In [29]:
numeric_cols = ['SibSp', 'Parch', 'Fare']
categorical_cols = ['Pclass', 'Sex', 'AgeGroup', 'Cabin', 'Embarked']
input_cols = numeric_cols + categorical_cols

In [30]:
raw_df[input_cols].isna().sum()

SibSp       0
Parch       0
Fare        0
Pclass      0
Sex         0
AgeGroup    0
Cabin       0
Embarked    0
dtype: int64

In [31]:
inputs = raw_df[input_cols]
targets = raw_df[target_col]

In [32]:
inputs

,SibSp,Parch,Fare,Pclass,Sex,AgeGroup,Cabin,Embarked
0,1,0,7.2500,3,male,Adult,UNK,S
1,1,0,71.2833,1,female,OldAdult,C85,C
2,0,0,7.9250,3,female,Adult,UNK,S
3,1,0,53.1000,1,female,OldAdult,C123,S
4,0,0,8.0500,3,male,OldAdult,UNK,S
...,...,...,...,...,...,...,...,...
886,0,0,13.0000,2,male,Adult,UNK,S
887,0,0,30.0000,1,female,Adult,B42,S
888,1,2,23.4500,3,female,Adult,UNK,S
889,0,0,30.0000,1,male,Adult,C148,C


In [33]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
scaler = MinMaxScaler().fit(inputs[numeric_cols])

In [35]:
inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])

<ipython-input-35-20aa88ed5fa6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])


In [36]:
inputs[numeric_cols].describe()

,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000
mean,0.065376,0.063599,0.062858
std,0.137843,0.134343,0.096995
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.015440
50%,0.000000,0.000000,0.028213
75%,0.125000,0.000000,0.060508
max,1.000000,1.000000,1.000000


In [37]:
from sklearn.preprocessing import OneHotEncoder

In [38]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [39]:
encoder.fit(inputs[categorical_cols])

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [40]:
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
encoded_cols

['Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Sex_female',
 'Sex_male',
 'AgeGroup_Adult',
 'AgeGroup_Baby',
 'AgeGroup_Child',
 'AgeGroup_Old',
 'AgeGroup_OldAdult',
 'AgeGroup_Teenager',
 'Cabin_A10',
 'Cabin_A14',
 'Cabin_A16',
 'Cabin_A19',
 'Cabin_A20',
 'Cabin_A23',
 'Cabin_A24',
 'Cabin_A26',
 'Cabin_A31',
 'Cabin_A32',
 'Cabin_A34',
 'Cabin_A36',
 'Cabin_A5',
 'Cabin_A6',
 'Cabin_A7',
 'Cabin_B101',
 'Cabin_B102',
 'Cabin_B18',
 'Cabin_B19',
 'Cabin_B20',
 'Cabin_B22',
 'Cabin_B28',
 'Cabin_B3',
 'Cabin_B30',
 'Cabin_B35',
 'Cabin_B37',
 'Cabin_B38',
 'Cabin_B39',
 'Cabin_B4',
 'Cabin_B41',
 'Cabin_B42',
 'Cabin_B49',
 'Cabin_B5',
 'Cabin_B50',
 'Cabin_B51 B53 B55',
 'Cabin_B57 B59 B63 B66',
 'Cabin_B58 B60',
 'Cabin_B69',
 'Cabin_B71',
 'Cabin_B73',
 'Cabin_B77',
 'Cabin_B78',
 'Cabin_B79',
 'Cabin_B80',
 'Cabin_B82 B84',
 'Cabin_B86',
 'Cabin_B94',
 'Cabin_B96 B98',
 'Cabin_C101',
 'Cabin_C103',
 'Cabin_C104',
 'Cabin_C106',
 'Cabin_C110',
 'Cabin_C111',
 'Cabin_C118',
 'Cabin_C12

In [41]:
encoded_df = pd.DataFrame(data=encoder.transform(inputs[categorical_cols]), columns=encoded_cols)
encoded_df.index = inputs.index
encoded_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,AgeGroup_Adult,AgeGroup_Baby,AgeGroup_Child,AgeGroup_Old,AgeGroup_OldAdult,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_UNK,Embarked_C,Embarked_Q,Embarked_S,Embarked_U
0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
888,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
889,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [42]:
inputs = pd.concat([inputs, encoded_df], axis=1).copy()

In [43]:
inputs = inputs[numeric_cols+encoded_cols]
inputs

,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,AgeGroup_Adult,AgeGroup_Baby,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_UNK,Embarked_C,Embarked_Q,Embarked_S,Embarked_U
0,0.125,0.000000,0.014151,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.125,0.000000,0.139136,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.000,0.000000,0.015469,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.125,0.000000,0.103644,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000,0.000000,0.015713,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.000,0.000000,0.025374,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,0.000,0.000000,0.058556,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
888,0.125,0.333333,0.045771,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
889,0.000,0.000000,0.058556,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [44]:
from sklearn.model_selection import train_test_split

In [45]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs, targets,
                                                                        test_size=0.2,
                                                                        random_state=42)

In [46]:
train_inputs.shape

(712, 166)

In [47]:
val_inputs.shape

(179, 166)

In [48]:
from xgboost import XGBClassifier

In [49]:
from sklearn.metrics import accuracy_score

In [50]:
model = XGBClassifier(n_jobs=-1, random_state=42, n_estimators=20, max_depth=5)

In [51]:
model.fit(train_inputs, train_targets)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [52]:
preds = model.predict(train_inputs)
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,

In [53]:
accuracy_score(train_targets, preds)

0.8820224719101124

In [54]:
val_preds = model.predict(val_inputs)

In [55]:
accuracy_score(val_targets, val_preds)

0.8100558659217877

In [56]:
def test_params(**params):
    model = XGBClassifier(n_jobs=-1, random_state=42, **params)
    model.fit(train_inputs, train_targets)
    train_acc = accuracy_score(train_targets, model.predict(train_inputs))
    val_acc = accuracy_score(val_targets, model.predict(val_inputs))
    print('train_acc: {}, val_acc: {}'.format(train_acc, val_acc))

In [57]:
test_params(n_estimators=20, max_depth=4)

train_acc: 0.8735955056179775, val_acc: 0.8044692737430168


In [58]:
final_model = XGBClassifier(n_jobs=-1, random_state=42,
                            learning_rate=0.1, n_estimators=30,
                            max_depth=6, sub_sample=0.9,
                            booster='gbtree')

In [59]:
final_model.fit(train_inputs, train_targets)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:08:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "sub_sample" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=30, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [60]:
final_model.score(train_inputs, train_targets)

0.8806179775280899

In [61]:
final_model.score(val_inputs, val_targets)

0.8044692737430168

In [62]:
test_df[numeric_cols].isna().sum()

SibSp    0
Parch    0
Fare     1
dtype: int64

In [63]:
test_df.fillna({'Age': avg_age}, inplace=True)

In [64]:
avg_fare = raw_df['Fare'].mean()

In [65]:
test_df.fillna({'Fare': avg_fare}, inplace=True)

In [66]:
test_df[numeric_cols].isna().sum()

SibSp    0
Parch    0
Fare     0
dtype: int64

In [67]:
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])

In [68]:
test_df[numeric_cols].describe()

,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000
mean,0.055921,0.065391,0.069524
std,0.112095,0.163571,0.108994
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.015412
50%,0.000000,0.000000,0.028213
75%,0.125000,0.000000,0.061484
max,1.000000,1.500000,1.000000


In [69]:
test_df['AgeGroup'] = pd.cut(test_df['Age'],
                            bins=[-1,5,14,18,30,60,100],
                            labels=['Baby','Child','Teenager','Adult','OldAdult','Old'])

In [70]:
test_df[categorical_cols].isna().sum()

Pclass        0
Sex           0
AgeGroup      0
Cabin       327
Embarked      0
dtype: int64

In [71]:
test_df.fillna({'Cabin': 'UNK'}, inplace=True)

In [72]:
test_df[categorical_cols].isna().sum()

Pclass      0
Sex         0
AgeGroup    0
Cabin       0
Embarked    0
dtype: int64

In [73]:
encoded_df = pd.DataFrame(data=encoder.transform(test_df[categorical_cols]), columns=encoded_cols)
encoded_df.index = test_df.index
test_df = pd.concat([test_df, encoded_df], axis=1).copy()

In [74]:
test_inputs = test_df[numeric_cols+encoded_cols]
test_inputs

,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,AgeGroup_Adult,AgeGroup_Baby,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_UNK,Embarked_C,Embarked_Q,Embarked_S,Embarked_U
0,0.000,0.000000,0.015282,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.125,0.000000,0.013663,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.000,0.000000,0.018909,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.000,0.000000,0.016908,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.125,0.166667,0.023984,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.000,0.000000,0.015713,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
414,0.000,0.000000,0.212559,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
415,0.000,0.000000,0.014151,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
416,0.000,0.000000,0.015713,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [75]:
test_preds = final_model.predict(test_inputs)

In [76]:
test_preds

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [77]:
accuracy_score(sub_df['Survived'], test_preds)

0.8827751196172249

In [78]:
sub_df['Survived'] = test_preds

In [79]:
sub_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [80]:
sub_df.to_csv('titanic_data/submission1.csv', index=None)

In [81]:
from sklearn.ensemble import RandomForestClassifier

In [82]:
base_rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)

In [83]:
base_rf_model.fit(train_inputs, train_targets)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [84]:
base_rf_model.score(train_inputs, train_targets)

0.9536516853932584

In [85]:
base_rf_model.score(val_inputs, val_targets)

0.8379888268156425

In [86]:
def test_params_rf(**params):
    model = RandomForestClassifier(n_jobs=-1, random_state=42, **params)
    model.fit(train_inputs, train_targets)
    train_acc = model.score(train_inputs, train_targets)
    val_acc = model.score(val_inputs, val_targets)
    return {'train_acc': train_acc, 'val_acc': val_acc}

In [87]:
final_rf_model = RandomForestClassifier(n_jobs=-1, random_state=42,
                                        n_estimators=30, max_depth=25,
                                        max_features='log2', max_leaf_nodes=2**5)

In [88]:
final_rf_model.fit(train_inputs, train_targets)

RandomForestClassifier(max_depth=25, max_features='log2', max_leaf_nodes=32,
                       n_estimators=30, n_jobs=-1, random_state=42)

In [89]:
final_rf_model.score(train_inputs, train_targets)

0.8623595505617978

In [90]:
final_rf_model.score(val_inputs, val_targets)

0.8044692737430168

In [91]:
test_preds = final_rf_model.predict(test_inputs)

In [92]:
accuracy_score(sub_df['Survived'], test_preds)

0.9330143540669856

In [93]:
test_preds

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [94]:
sub_df['Survived'] = test_preds

In [95]:
sub_df.to_csv('titanic_data/submission2.csv', index=None)

## Feed Forward NN Model

In [96]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

In [97]:
def accuracy(output, target):
    _, pred = torch.max(output, dim=1)
    return torch.tensor(torch.sum(pred == target).item()/len(target))

In [98]:
class BaseModel(nn.Module):
    def training_step(self, batch):
        inputs, targets = batch
        outputs = self(inputs)
        probs = torch.sigmoid(outputs[:,0])
        loss = F.binary_cross_entropy(probs, targets)
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        outputs = self(inputs)
        probs = torch.sigmoid(outputs[:,0])
        loss = F.binary_cross_entropy(probs, targets)
        preds = (probs > 0.5).int()
        acc = accuracy_score(targets, preds)
        return {'val_loss': loss.item(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = np.mean(batch_losses)
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = np.mean(batch_accs)
        return {'val_loss': epoch_loss, 'val_acc': epoch_acc}

    def epoch_end(self, epoch, result):
        print('Epoch {}, val_loss: {:4f}, val_acc: {:4f}'.format(epoch,
                                                                 result['val_loss'],
                                                                 result['val_acc']))

In [99]:
inputs.shape, targets.shape

((891, 166), (891,))

In [100]:
input_tensors = torch.tensor(inputs.values, dtype=torch.float32)
target_tensors = torch.tensor(targets.values, dtype=torch.float32)

In [101]:
input_tensors.shape, target_tensors.shape

(torch.Size([891, 166]), torch.Size([891]))

In [102]:
from torch.utils.data import TensorDataset, DataLoader

In [103]:
raw_ds = TensorDataset(input_tensors, target_tensors)
raw_ds[0]

(tensor([0.1250, 0.0000, 0.0142, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [104]:
random_seed = 42
torch.manual_seed(random_seed)

In [105]:
val_size = 170
train_size = len(raw_ds) - val_size
from torch.utils.data import random_split
train_ds, val_ds = random_split(raw_ds, [train_size, val_size])

In [106]:
len(train_ds), len(val_ds)

(721, 170)

In [107]:
batch_size = 32

In [108]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size*2)

In [109]:
class TwoLayerModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(166,64)
        self.linear2 = nn.Linear(64,1)

    def forward(self, xb):
        out = self.linear1(xb)
        out = F.relu(out)
        out = self.linear2(out)
        return out

In [110]:
two_layer_model = TwoLayerModel()

In [111]:
def evaluate(model, val_dl):
    outputs = [model.validation_step(batch) for batch in val_dl]
    return model.validation_epoch_end(outputs)

In [112]:
def fit(epochs, lr, model, train_dl, val_dl, opt_func=torch.optim.SGD):
    history = []
    opt = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        for batch in train_dl:
            loss = model.training_step(batch)
            loss.backward()
            opt.step()
            opt.zero_grad()
        result = evaluate(model, val_dl)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [113]:
history = [evaluate(two_layer_model, val_dl)]
history

[{'val_loss': 0.6769498785336813, 'val_acc': 0.7145337301587302}]

In [114]:
history += fit(5, 0.1, two_layer_model, train_dl, val_dl)

Epoch 0, val_loss: 0.611826, val_acc: 0.709325
Epoch 1, val_loss: 0.552783, val_acc: 0.724950
Epoch 2, val_loss: 0.500501, val_acc: 0.787698
Epoch 3, val_loss: 0.462593, val_acc: 0.853175
Epoch 4, val_loss: 0.438047, val_acc: 0.826885


In [115]:
history += fit(5, 0.07, two_layer_model, train_dl, val_dl)

Epoch 0, val_loss: 0.417849, val_acc: 0.826885
Epoch 1, val_loss: 0.413241, val_acc: 0.813740
Epoch 2, val_loss: 0.411934, val_acc: 0.826885
Epoch 3, val_loss: 0.405678, val_acc: 0.826885
Epoch 4, val_loss: 0.398311, val_acc: 0.826885


In [116]:
history += fit(5, 0.05, two_layer_model, train_dl, val_dl)

Epoch 0, val_loss: 0.397029, val_acc: 0.826885
Epoch 1, val_loss: 0.397413, val_acc: 0.826885
Epoch 2, val_loss: 0.405444, val_acc: 0.832093
Epoch 3, val_loss: 0.392340, val_acc: 0.832093
Epoch 4, val_loss: 0.392062, val_acc: 0.832093


In [117]:
test_ds = TensorDataset(torch.tensor(test_inputs.values, dtype=torch.float32))
test_dl = DataLoader(test_ds, batch_size*2)

In [118]:
def predict(test_dl, model):
    all_preds = []
    for batch in test_dl:
        inputs, = batch
        outputs = model(inputs)
        probs = torch.sigmoid(outputs[:,0])
        preds = (probs > 0.5).int()
        all_preds += list(preds.numpy())
    return all_preds

In [119]:
test_preds = predict(test_dl, two_layer_model)
test_preds

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [120]:
sub_df['Survived'] = test_preds
sub_df.to_csv('titanic_data/submission3.csv', index=None)

In [121]:
class ThreeLayerModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(166, 128)
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64, 1)

    def forward(self, xb):
        out = F.relu(self.linear1(xb))
        out = F.relu(self.linear2(out)) + out
        out = F.relu(self.linear3(out))
        out = self.linear4(out)
        return out

In [122]:
three_layer_model = ThreeLayerModel()

In [123]:
history = [evaluate(three_layer_model, val_dl)]

In [124]:
history

[{'val_loss': 0.6916847030321757, 'val_acc': 0.7041170634920636}]

In [125]:
history += fit(3, 0.1, three_layer_model, train_dl, val_dl)

Epoch 0, val_loss: 0.630561, val_acc: 0.709325
Epoch 1, val_loss: 0.587518, val_acc: 0.709325
Epoch 2, val_loss: 0.547781, val_acc: 0.766865


In [126]:
history += fit(3, 0.05, three_layer_model, train_dl, val_dl)

Epoch 0, val_loss: 0.505332, val_acc: 0.798115
Epoch 1, val_loss: 0.465999, val_acc: 0.834821
Epoch 2, val_loss: 0.449492, val_acc: 0.847966


In [127]:
test_preds = predict(test_dl, three_layer_model)
test_preds

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [128]:
sub_df['Survived'] = test_preds
sub_df.to_csv('titanic_data/submission5.csv', index=None)

## ResNET

In [129]:
input_tensors = input_tensors.reshape(-1, 1, 166)
input_tensors

tensor([[[0.1250, 0.0000, 0.0142,  ..., 0.0000, 1.0000, 0.0000]],

        [[0.1250, 0.0000, 0.1391,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0155,  ..., 0.0000, 1.0000, 0.0000]],

        ...,

        [[0.1250, 0.3333, 0.0458,  ..., 0.0000, 1.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0586,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0151,  ..., 1.0000, 0.0000, 0.0000]]])

In [130]:
raw_ds = TensorDataset(input_tensors, target_tensors)
val_size = 170
train_size = len(raw_ds) - val_size
from torch.utils.data import random_split
train_ds, val_ds = random_split(raw_ds, [train_size, val_size])

In [131]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size*2)

In [132]:
for batch in train_dl:
    inputs, targets = batch
    print(inputs.shape)
    conv1d = nn.Conv1d(1,64,kernel_size=3,padding=1)
    outs = conv1d(inputs)
    print(outs.shape)
    break

torch.Size([32, 1, 166])
torch.Size([32, 64, 166])


In [133]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm1d(out_channels),
              nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool1d(2))
    return nn.Sequential(*layers)

In [160]:
class ResNetModel(BaseModel):
    def __init__(self):
        super().__init__()

        self.prep = conv_block(1,64)
        self.layer1 = conv_block(64,128,pool=True)
        self.res1 = nn.Sequential(conv_block(128,128), conv_block(128,128))

        self.layer2 = conv_block(128,256,pool=True)
        self.layer3 = conv_block(256,512,pool=True)
        self.res2 = nn.Sequential(conv_block(512,512), conv_block(512,512))

        self.classifier = nn.Sequential(
            nn.MaxPool1d(4),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(2560,1)
        )

    def forward(self, xb):
        out = self.prep(xb)
        out = self.layer1(out)
        out = self.res1(out) + out
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [136]:
@torch.no_grad()
def evaluate(model: BaseModel, val_dl: DataLoader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_dl]
    return model.validation_epoch_end(outputs)

In [137]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [138]:
def fit(epochs: int, max_lr: float, model: BaseModel,
        train_dl: DataLoader, val_dl: DataLoader, opt_func = torch.optim.SGD,
        weight_decay: float=0, grad_clip=None):
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_dl))
    for epoch in range(epochs):
        # Train
        model.train()
        for batch in train_dl:
            loss = model.training_step(batch)
            loss.backward()
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            sched.step()
        # Validation
        result = evaluate(model, val_dl)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [166]:
model = ResNetModel()

In [167]:
history = [evaluate(model, val_dl)]

In [168]:
history

[{'val_loss': 0.6906862656275431, 'val_acc': 0.5520833333333334}]

In [169]:
history += fit(10, 0.01, model, train_dl, val_dl,
               opt_func = torch.optim.Adam,
               weight_decay = 1e-4, grad_clip=0.1)

Epoch 0, val_loss: 0.602885, val_acc: 0.646329
Epoch 1, val_loss: 0.990797, val_acc: 0.555308
Epoch 2, val_loss: 1.156834, val_acc: 0.732887
Epoch 3, val_loss: 2.980055, val_acc: 0.567708
Epoch 4, val_loss: 0.756798, val_acc: 0.743552
Epoch 5, val_loss: 0.574840, val_acc: 0.816964
Epoch 6, val_loss: 0.539005, val_acc: 0.785714
Epoch 7, val_loss: 0.559844, val_acc: 0.759425
Epoch 8, val_loss: 0.543747, val_acc: 0.767113
Epoch 9, val_loss: 0.562636, val_acc: 0.803819


In [170]:
test_tensors = torch.tensor(test_inputs.values, dtype=torch.float32).reshape(-1,1,166)
test_ds = TensorDataset(test_tensors)
test_dl = DataLoader(test_ds, batch_size*2)

In [171]:
test_preds = predict(test_dl, model)
test_preds

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,


In [172]:
sub_df['Survived'] = test_preds
sub_df.to_csv('titanic_data/submission6.csv', index=None)